Романовский А.Н. 28.09.2022  
#### Исследование потоков и процессов при помощи парсинга 
____

### Задание:
Реализовать с использованием потоков и процессов скачивание файлов из интернета. 

Список файлов для скачивания подготовить самостоятельно (например изображений, не менее 100 изображений или других объектов). 

Сравнить производительность с последовательным методом. 


Сравнивть производительность Thread и multiprocessing решений.

Попробовать подобрать оптимальное число потоков/процессов. 

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import os
import shutil
import concurrent
import multiprocessing

#### Использование одного потока и одного процесса 

In [2]:
page_number = 1
img_list = [] #лист с ссылками изображений 
for page_number in range(1, 7):
    response = requests.get(f'https://zastavok.net/{page_number}') #запрс на страницу 
    if response.status_code == 200: #ответ 
        soup = BeautifulSoup(response.text, features="html.parser")
        find_class = soup.find('div', class_="block-photo") #ищем основной блок с фото
        all_image = find_class.find_all('div', class_="short_full" ) # ищем все теги в основном блоке

        for image in all_image:  #перебираем список тегов
            image_url = image.find('img', class_="short_prev_img").get('src') #в нем находим по тегу и отправляем запрос на ссылку с привью картинкой

            img_list.append(image_url) #добавляем в лист
            if len(img_list) == 100:
                    break   
                        
    else:
        print('try again')

In [3]:
print(len(img_list), 'links downloaded') #размер листа с изображениями 

100 links downloaded


In [4]:
def download(link): # функция сохраняет изображения 
    link_name = requests.get(f'https://zastavok.net/{link}') #запрос на ссылку 
    with open(f'{path}{img_list.index(link)}.jpg', 'wb') as file:   #запись 
             file.write(link_name.content)

In [5]:
path = ('D:/image/')            #создаем папку 
try: 
    os.mkdir(path)
except FileExistsError:
    shutil.rmtree (path) #проверка на ошибку  
    os.mkdir(path)

start = time.time()  

for link in img_list:  #циклом запишем в однопоточном режиме 
    download(link)   

end = time.time()
print('Одиночная загрузка составляет ' +str(round(end - start )) + 'c')
shutil.rmtree(path)

Одиночная загрузка составляет 27c


#### Использование многопоточности при помощи ThreadPoolExecutor

In [7]:
for multithreaded in range(2, 18): #в цикле будем менять количество потоков 
    path = ('D:/image/')            
    try: 
        os.mkdir(path)
    except FileExistsError:
        shutil.rmtree (path)
        os.mkdir(path)
    start = time.time()
    with concurrent.futures.ThreadPoolExecutor(multithreaded) as executor:
        executor.map(download, img_list)   #каждый вызов  download итерируется по img_list и выполняетя в отдельном потоке 
    end = time.time()
    print(f'{multithreaded}, потока выполняют задачу за ' +str(round(end - start ))  + 'c') 

2, потока выполняют задачу за 13c
3, потока выполняют задачу за 9c
4, потока выполняют задачу за 7c
5, потока выполняют задачу за 6c
6, потока выполняют задачу за 5c
7, потока выполняют задачу за 4c
8, потока выполняют задачу за 5c
9, потока выполняют задачу за 4c
10, потока выполняют задачу за 4c
11, потока выполняют задачу за 4c
12, потока выполняют задачу за 4c
13, потока выполняют задачу за 3c
14, потока выполняют задачу за 3c
15, потока выполняют задачу за 2c
16, потока выполняют задачу за 2c
17, потока выполняют задачу за 3c


#### Использование многопроцесеррности при помощи multiprocessing

In [ ]:
path = ('D:/image/')
if __name__ == "__main__":
    for process in range(1,7): #в цикле будем менять количество процессов 
        try: 
            os.mkdir(path)
        except FileExistsError:
            shutil.rmtree (path)
            os.mkdir(path)
        start = time.time()
        with multiprocessing.Pool(process) as pool:
            pool.map(download, img_list)  #каждый вызов  download итерируется по img_list и выполняетя в отдельном процессе

        end = time.time()
        print(f'{process} процессов выполняют задачу за' +str(round(end - start )) + 'c')


____
### Вывод:
#### при выполнении задачи, были использованы модули multiprocessing и concurrent
* одинопоточная загрузка составила 27с
* при использовании многопоточного режима, лучши результат оказался 15-16 потоков, за 2с
* при многопроцессорной загрузке лучший показатель 5-6 потоков за 9с
* в данной задаче, самым быстрым вариантом является многопоточный режим